# ML model generation

This notebook creates and trains a simple fully connected model using MNIST dataset. At the end, the model is exported as `model.bin`.

In [1]:
#|export
import numpy as np
import torchvision
from torchvision import transforms, datasets
import torch
from torch import nn
import matplotlib.pyplot as plt
import struct
import os

import torchvision.models as models
from torchvision.models import ResNet18_Weights
from model import ResNetMnist # my model
from export import export_model
from export import export_modelq8

In [2]:
#|export
def generate_dataloaderImagenette(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224),        
        transforms.ToTensor(),             
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
    ])
    #data = torchvision.datasets.Imagenette("./data", download=True)
    traindataset = datasets.ImageFolder(root='./data/imagenette2/train', transform=transform)
    testdataset = datasets.ImageFolder(root='./data/imagenette2/val', transform=transform)

    trainloader = torch.utils.data.DataLoader(traindataset, batch_size=batch_size, shuffle=True)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=batch_size, shuffle=False)
    
    return trainloader, testloader

def generate_dataloaderMNIST(batch_size=32):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    
    trainset = torchvision.datasets.MNIST("./data", train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    
    testset = torchvision.datasets.MNIST("./data", train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    return trainloader, testloader

In [5]:
#|export
def test_model(model, testloader):
    loss_fn = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        vloss = 0.
        correct = 0.

        # Fetch the first batch
        for inputs, targets in testloader:
            outputs = model(inputs)
            vloss += loss_fn(outputs, targets).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted==targets).sum().item()
    
    return vloss/len(testloader),  correct/len(testloader.dataset)


def train_model(model):  
    # training
    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=0.001)
    trainloader, testloader = generate_dataloaderMNIST()

    for epoch in range(1):

        model.train()
        tloss = 0
        for inputs, targets in trainloader:
            opt.zero_grad()
            out = model(inputs)
            loss = loss_fn(out, targets)
            loss.backward()
            tloss += loss.item()
            opt.step()

        tloss = tloss/len(trainloader)
        vloss, correct = test_model(model, testloader)

        print('LOSS train {} valid {} accuracy {:.5f}'.format(tloss, vloss, correct))
    torch.save(model, "modelMNIST.pt")

In [6]:
model = ResNetMnist()
train_model(model)

LOSS train 0.10557933791515728 valid 0.02860270043334421 accuracy 0.98960


In [8]:
print(model)

ResNetMnist(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (group1): GroupOfBlocks(
    (group): Sequential(
      (0): Block(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (skip_connection): Identity()
        (act): ReLU()
      )
      (1): Block(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): 

In [114]:
modelr = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

In [140]:
torch.save(modelr, 'model.pt')
modelr = torch.load('model.pt')

In [142]:
_, testloader = generate_dataloaderMNIST()
loss, acc = test_model(torch.load("model.pt"), testloader)
print("Accuracy: {:.2f} %".format(100*acc))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


KeyboardInterrupt: 

In [61]:
import torchvision.models as models
from torchvision.models import ResNet18_Weights

modelr = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

In [62]:
print(modelr)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
model = torch.load("modelMNIST.pt")
print(model)

ResNetMnist(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (group1): GroupOfBlocks(
    (group): Sequential(
      (0): Block(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (skip_connection): Identity()
        (act): ReLU()
      )
      (1): Block(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding

In [3]:
export_model(model)

wrote model.bin


In [5]:
conv_layers = [layer for layer in model.modules() if isinstance(layer, torch.nn.Conv2d)]
bn_layers = [layer for layer in model.modules() if isinstance(layer, torch.nn.BatchNorm2d)]
print(conv_layers[0].kernel_size)

(5, 5)


In [6]:
def im2col(input_data, kernel_size, stride, padding):
    # Add padding to the input data
    padded_data = np.pad(input_data, 
                         ((padding, padding), (padding, padding), (0, 0)), 
                         mode='constant')

    # Calculate the output dimensions
    out_height = (input_data.shape[0] + 2*padding - kernel_size) // stride + 1
    out_width = (input_data.shape[1] + 2*padding - kernel_size) // stride + 1

    # Initialize the output matrix
    col = np.zeros((kernel_size, kernel_size, input_data.shape[2], out_height, out_width))

    # Fill the output matrix
    for y in range(kernel_size):
        y_max = y + stride * out_height
        for x in range(kernel_size):
            x_max = x + stride * out_width
            col[y, x, :, :, :] = padded_data[y:y_max:stride, x:x_max:stride, :]

    # Reshape and return
    col = col.transpose(3, 4, 2, 0, 1).reshape(out_height * out_width, -1)
    return col


In [7]:
#col = im2col(conv_layers[0].weight.detach(), conv_layers[0].kernel_size[0], conv_layers[0].stride, conv_layers[0].padding[0])
print(conv_layers[0].padding[0])
unfold = torch.nn.Unfold(kernel_size=conv_layers[0].kernel_size[0], stride=conv_layers[0].stride, padding=conv_layers[0].padding[0])
col = unfold(conv_layers[0].weight.detach())
print(bn_layers[3].bias)
#print(col[0])

2
Parameter containing:
tensor([-0.0499, -0.0008, -0.0515, -0.0151, -0.0864, -0.0439, -0.0378, -0.0170,
        -0.0254, -0.0166, -0.0085, -0.0161, -0.0053, -0.0576, -0.0940, -0.0239,
        -0.0396, -0.0609, -0.0556, -0.0049, -0.0682, -0.0565, -0.0401, -0.0474,
         0.0032, -0.0222, -0.0807, -0.0333, -0.0486, -0.0317, -0.0060, -0.0279,
         0.0092, -0.0661, -0.0780, -0.0298, -0.0635, -0.0125, -0.0466, -0.0136,
        -0.0908, -0.0045, -0.0535,  0.0005, -0.0341,  0.0093, -0.0396, -0.0222,
        -0.0413, -0.0508, -0.0523,  0.0010, -0.0185, -0.0681, -0.0579,  0.0007,
        -0.0106, -0.0309, -0.0533, -0.0214, -0.0542, -0.0072, -0.0115, -0.0345],
       requires_grad=True)


In [9]:
print(bn_layers[3])

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
